#  Tutorial：Load 3D-Front Dataset into Maya 2019/2020/2022

In [1]:
# the tool kit to parsing data
from maya import * 

# the controller (as a socket client) to control maya 
from controller import MayaController

# import some args
from param import args

# Step 1: Open maya server

- Now we are ready to open [Maya](https://www.autodesk.com/products/maya/features)
- In Maya, open [script editor](https://knowledge.autodesk.com/support/maya/learn-explore/caas/CloudHelp/cloudhelp/2020/ENU/Maya-Scripting/files/GUID-7C861047-C7E0-4780-ACB5-752CD22AB02E-htm.html)
- And type the following command into it.

```python
# Open a command port with the default name "mayaCommand".
import maya.cmds as cmds
cmds.commandPort(n="localhost:12345")
```




# Step 2: Open Client

In [4]:
# Connect Maya
# import maya.cmds as cmds
# cmds.commandPort(n="localhost:12345")
mc = MayaController(PORT=12345)
mc.SendPythonCommand("from maya.api import OpenMaya")

'\n\x00'

### Specify your 3D-Front data path here

FOr example, the file structure is like this

```
├── 3D-FRONT
    ├── 3D-FRONT
        ├── 00ad8345-45e0-45b3-867d-4a3c88c2517a.json
        ├── .......json

    ├── 3D-FRONT-texture
    └── 3D-FUTURE-model
        ├── 00ad8345-45e0-45b3-867d-4a3c88c2517a
        ├── ......
```

In [7]:
### directory of meshes
shapeLocalSource = "E://Temp/3Dfront/3D-FUTURE-model/"
houseLayoutSource = "E://Temp/3Dfront/3D-FRONT/"
allHouseLayouts = os.listdir(houseLayoutSource)


In [8]:
### Set new scene maya
mc.SetNewScene()
# IN_METER = False
# if IN_METER:
#     mc.SendCommand('currentUnit -linear "meter";')

### Specify your housing index

In [9]:
houseLayoutIndex = 0

In [10]:
### Load room layout json
houseLayoutFile = os.path.join(houseLayoutSource, allHouseLayouts[houseLayoutIndex])
with open(houseLayoutFile) as f: sceneDict = json.load(f)

In [11]:
print(sceneDict.keys())

dict_keys(['uid', 'jobid', 'design_version', 'code_version', 'north_vector', 'furniture', 'mesh', 'material', 'lights', 'extension', 'scene', 'groups', 'materialList', 'version'])


In [12]:
### Extract info
instance_table = create_instance_table(sceneDict)
mesh_table = create_mesh_table(sceneDict)
material_table = create_material_table(sceneDict)
furniture_table = create_furniture_table(sceneDict)

E:\Blender\maya.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return {key: np.array(value) for (key, value) in _.items()}


In [13]:
### Join information tables
mesh_material = join(mesh_table, material_table, 'material_id', 'id', 'material_')
mesh_all = join(mesh_material, instance_table, 'id', 'ref', 'instance_')
furniture_all = join(furniture_table, instance_table, 'id', 'ref', 'instance_')

E:\Blender\maya.py:152: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  columns = np.array(result).T.tolist()


In [17]:
# get furniture list
furniture_list = list(zip(furniture_all['jid'], furniture_all['position'], furniture_all['rotation'], furniture_all['scale']))

In [19]:
# create an empty group node with no children
mc.SendPythonCommand("cmds.group( em=True, name='structure')")
mc.SendPythonCommand("cmds.group( em=True, name='ceilings')")
mc.SendPythonCommand("cmds.group( em=True, name='floors')")
mc.SendPythonCommand("cmds.group( em=True, name='windows')")
mc.SendPythonCommand("cmds.group( em=True, name='doors')")

'doors\n\x00'

#### Import Mesh

In [20]:
# load mesh into maya
merge_list = []
for index, mesh in enumerate(zip(mesh_all['id'], mesh_all['type'], mesh_all['xyz'], 
                                     mesh_all['normal'], mesh_all['uv'], mesh_all['face'], mesh_all['color'])):             
    
    # Select the current set of objects
    mc.SendPythonCommand('cmds.select(all = True, hierarchy = True)')
    mc.SendPythonCommand('currentObjs = cmds.ls(selection = True )')

    id_, type_, xyz, normal, uv, face, color = mesh
    xyz = np.array(xyz.tolist()).astype(np.float)
    normal = np.array(normal.tolist()).astype(np.float)
    uv = np.array(uv.tolist()).astype(np.float)
    face = np.array(face.tolist()).astype(np.int)

    vertices = xyz.T.tolist()
    faces = face.T.tolist()
    
    vertices, faces = fix_vertices_and_faces(vertices, faces)
    # print(vertices,faces)
    
    mc.SendPythonCommand("meshFn = OpenMaya.MFnMesh()")
    
#     time.sleep(0.1)

    code_multiLine = """
vertices = []
polygonFaces = []
polygonConnects = [] 
# print(vertices)
    """.replace("\n",r"\n")  

    message = 'python("{}")'.format(code_multiLine)
    mc.SendCommand(message)

#     time.sleep(0.1)

    for v in vertices:
        x,y,z = v
        code_multiLine = f"""vertices.append(OpenMaya.MPoint({x}, {y}, {z}))""".replace("\n",r"\n")  
        #print(code_multiLine)
        message = 'python("{}")'.format(code_multiLine)
        mc.SendCommand(message)

    for f in faces:
        code_multiLine = f"""polygonFaces.append({len(f)})"""
        #code_multiLine = f"""polygonConnects += {f}"""
        # print(code_multiLine)
        message = 'python("{}")'.format(code_multiLine)
        mc.SendCommand(message)

    for f in faces:
        # code_multiLine = f"""polygonFaces.append({len(f)})"""
        code_multiLine = f"""polygonConnects += {f}"""
        # print(code_multiLine)
        message = 'python("{}")'.format(code_multiLine)
        mc.SendCommand(message)

    # Select and rename the newly added mesh
    mc.SendPythonCommand("meshFn.create(vertices, polygonFaces, polygonConnects)")
    mc.SendPythonCommand(f"cmds.sets(meshFn.name(), edit=True, forceElement=\\'initialShadingGroup\\')")

    #mc.SendPythonCommand('cmds.select(all = True)')
    #mc.SendPythonCommand('cmds.select(currentObjs, deselect = True)')
    #mc.SendPythonCommand('newObjs = cmds.ls(selection = True, transforms = True )')
    mc.SendPythonCommand("show_name = cmds.listRelatives(meshFn.name(), fullPath=True, parent=True)[0]")
    mc.SendPythonCommand(f"cmds.rename(show_name,'{type_}_{index}')")
    
    # Hide object if it is a ceiling
    if "Ceiling" in type_:
        mc.SendPythonCommand(f"cmds.parent( '{type_}_{index}', 'ceilings' )")
        mc.SendPythonCommand("cmds.hide()")
    elif "Floor" in type_:
        # if len(vertices) < 20:
            # mc.SendCommand(f"select -r {type_}_{index};")
            # mc.SendCommand(retopology_cmd)
            #time.sleep(2)
        mc.SendPythonCommand(f"cmds.polyProjection('{type_}_{index}.f[0:]', type='Planar', md='y' )")
        mc.SendPythonCommand(f"cmds.parent( '{type_}_{index}', 'floors' )")
    elif "Window" in type_:
        mc.SendPythonCommand(f"cmds.parent( '{type_}_{index}', 'windows' )")
    else:
        mc.SendPythonCommand(f"cmds.parent( '{type_}_{index}', 'structure' )")
        merge_list.append(f'{type_}_{index}')
    # break

C:\Users\Yizhou Zhao\AppData\Local\Temp\ipykernel_19844\1513491567.py:11: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  xyz = np.array(xyz.tolist()).astype(np.float)
C:\Users\Yizhou Zhao\AppData\Local\Temp\ipykernel_19844\1513491567.py:12: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  normal = np.array(normal.tolist()).astype(np.float)
C:

#### Now your maya viewport should be like this

![maya_mesh](./imgs/maya_mesh.png)

#### Import furniture

In [21]:
mc.SendPythonCommand("cmds.group( em=True, name='furniture')")
mc.SendPythonCommand("cmds.group( em=True, name='lights')")

'lights\n\x00'

In [22]:
# furniture_all['scale']

In [23]:
for index, furniture in enumerate(zip(furniture_all['id'], furniture_all['jid'], furniture_all['position'], furniture_all['rotation'], furniture_all['scale'])): 
    fid, jid, position, rotation, scale = furniture
    raw_model_path = os.path.join(shapeLocalSource, jid, 'raw_model.obj').replace("\\","/")
    texture_path = os.path.join(shapeLocalSource, jid, 'texture.png').replace("\\","/")
    if  os.path.exists(raw_model_path) and os.path.exists(texture_path):
        fid = fid.split("/")[0]
        
          # If y position > 1, we see this as a light
        if position[1] > 1:
            mc.SendPythonCommand(f"cmds.pointLight(position=[{position[0]},{position[1]},{position[2]}], name = 'light_{fid}', intensity=20)")
            mc.SendPythonCommand(f"cmds.parent( 'light_{fid}', 'lights' )")
            
        # print(fid)
        mc.SendPythonCommand(f"cmds.file('{raw_model_path}', i=True, gr=True, gn='furniture_group', mergeNamespacesOnClash=True, namespace='component_{fid}')")
        mc.SetObjectWorldTransform('furniture_group', position)
        # print("rotation: ", euler_from_quaternion(*rotation), rotation)
        mc.SetObjectLocalRotation('furniture_group',np.rad2deg(euler_from_quaternion(*rotation)))
        #mc.SetObjectAttribute("furniture_group", "scaleX", )
        
        # print("scale", scale)
        mc.SetObjectAttribute("furniture_group", "scaleX", scale[0])
        mc.SetObjectAttribute("furniture_group", "scaleY", scale[1])
        mc.SetObjectAttribute("furniture_group", "scaleZ", scale[2])
        
        mc.SendPythonCommand(f"cmds.parent( 'furniture_group', 'furniture' )")
        mc.SendPythonCommand(f"cmds.rename('furniture_group', 'furniture_{fid}')") 

        # break
        
    

#### Now your maya viewport should be like this

![maya_furni](./imgs/maya_furni.png)